Vectorizado con métodos clásicos
================================

## Introducción

Un modelos de aprendizaje automático es, a groso modo, una función parametrizada `f(x)` que toma como entrada
un vector `x`, `n-dimensional`, y produce un vector de salida `m-dimensional`. Tal función puede ser simple (para un modelo lineal por ejemplo) o más compleja (como una red neuronal).

Cuando trabajamos con lenguaje natural, la mayoría de nuestros datos de entrada representarán características discretas y categóricas, ya sean palabras, letras o incluso utterancias (partes del discurso). La pregunta que nos haremos entonces es ¿Cómo codificamos esos datos categóricos de una manera que sea práctica para ser utilizada por un modelo de aprendizaje automático?

Discutiremos las opciones disponibles.

 - [One-hot encoding](#One-hot-encoding)
 - [Index-based encoding](#Index-based-encoding)
 - [Basados en frecuencias](#Basados-en-frecuencias)

 Utilizaremos el siguiente corpus para nuestros ejemplos:


In [1]:
corpus = ["El hielo es agua en estado sólido",
          "El hielo es uno de los cuatro estados naturales del agua",
          "El agua pura se congela a 0 grados",
          "El hielo es el nombre común del agua en estado sólido"]

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [2]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/vectorization/vectorization.txt --quiet --no-clobber
!pip install -r vectorization.txt

## One-hot encoding

Dado que el texto representará características discretas y categóricas, hace sentido pensar en utilizar métodos clásicos para este tipo de dato. **One-hot encoding** es una técnica sensilla que cosiste en representar las palabras con vectores de longitud igual al tamaño del vocabulario donde todas las posiciones son zero salvo la posición que corresponde al indice de la palabra en cuestión. Eso significa que las dimensiones de los vectores de entrada dependerá del tamaño del vocabulario y no del tamaño del cuerpo de texto. Este tipo de representación tiene la propiedad de que todas las palabras son igualmente relevantes para el modelo.

Si bien este método es sencillo de implementar, genera representaciones dispersas, que genera dificultades a la hora de procesarlos. Sin embargo esto no significa que esta forma de codificación no sea práctica. Veremos más adelante que esta técnica se puede utilizar para aprender representaciones más complejas como [embeddings](https://m72109.readthedocs.io/es/latest/nlp/vectorization/embeddings.html). En estas configuraciones, la entrada de la red neuronal está especificada como una colección de vectores one-hot.

## Index-based encoding

Es una técnica similar a `one-hot encoding` salvo que aqui los vectores están representados utilizando los valores numéricos correspondientes a los índices que ocupan cada palabra dentro del vocabulario. Es decir que cada palabra está codificada como un número entero. Esto hace que el tamaño del vector no dependa del tamaño del vocabulario.

En general, esta técnica no ofrece ninguna ventaja, pero se utiliza como una representación intermedia para implementar otras formas de vectorización. Por este motivo no se la suele referenciar como una técnica de **vectorización** sino que como un **diccionario de palabras** (ya que mapea *palabras* con *IDs únicos*) Veamos como utilizarla:

Primero crearemos nuestro vocabulario:

In [3]:
vocab = { i:w for w,i in enumerate(set(' '.join(corpus).split())) }

Utilizando el vocabulario, transformamos nuestros documentos en vectores:

In [4]:
vectors = [[vocab[w] for w in s.split(' ')] for s in corpus]

In [5]:
import pandas as pd

pd.DataFrame(vectors)

,0,1,2,3,4,5,6,7,8,9,10
0,8,14,16,7,1,19,0,NaN,NaN,NaN,NaN
1,8,14,16,9,21,3,22,5.0,4.0,20.0,7.0
2,8,7,15,2,6,10,12,11.0,NaN,NaN,NaN
3,8,14,16,13,18,17,20,7.0,1.0,19.0,0.0


## Basados en frecuencias

Se trata de una familia de métodos de los más ampliamente utilizado durante mucho tiempo para convertir texto en representaciones numéricas. En general, estos métodos representan el texto utilizando una lista de frecuencia de palabras, es decir, basandose de alguna forma en la frecuencia en la que aparecen.

Los métodos basados en bag-of-words tienen las siguientes limitaciones:

 - El orden de las palabras es ignorado
 - La frecuencia de la plabra no necesariamente encapsula la importancia
 - Las frecuencias marginales juegan un papel importante (relación entre files y columnas)

### Term frecuency

Utiliza un vector de longitud igual al tamaño del vocabulario, pero donde los valores corresponden a la frecuencia de la palabra w en el documento D. Las palabras más frecuentes tienen más relevancia.

$$TF = \frac {freq(w_i)} {len(doc)} $$

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0., max_df=1.)
vectors = vectorizer.fit_transform(corpus)

In [7]:
vectors = vectors.todense()

In [8]:
vectors.shape

(4, 20)

> ¿Que representa 20 en las dimensiones del vector?

In [10]:
import pandas as pd
import numpy as np

# Obtenemos todas las palabras del vocabulario
vocab = vectorizer.get_feature_names_out()
# Vectores de cada uno de los documentos
pd.DataFrame(vectors, columns=vocab)

,agua,común,congela,cuatro,de,del,el,en,es,estado,estados,grados,hielo,los,naturales,nombre,pura,se,sólido,uno
0,1,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,1,0
1,1,0,0,1,1,1,1,0,1,0,1,0,1,1,1,0,0,0,0,1
2,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0
3,1,1,0,0,0,1,2,1,1,1,0,0,1,0,0,1,0,0,1,0


### TF-IDF

Se trata de un método similar a Term Frecuency, pero cuyo objetivo es tratar de ajustar la frecuencia de la palabra en cada documento considerando que tan frecuente es dentro del corpus (dispersión). Dispersión justamente se refiere a que tan equitativamente las palabras están distribuidas entre los diferentes documentos del texto.

Este método considera que:

 - Cuanto más frecuente es una palabra en el corpus (más dispersa), más general es su significado.
 - Cuanto más centralizada está el uso de una palabra dentro de todo el corpus (baja dispersión), más probable es que la palabra represente un tópico puntual.

Para calcular el peso de cada palabra debemos obtener:

 - **TF:** La frecuencia del termino en el corpus.

 $$TF = \frac {freq(w_i)} {len(doc)} $$

 - **IDF:** La frecuencia (inversa) del termino en el documento.

$$IDF = 1 + log(\frac {len(corpus)} {freq(w_i, corpus)}) $$

Finalmente, `tfidf` se computa como la multiplicación de los dos terminos. Adicionalmente, se normaliza usando `L2`, es decir, la norma euclidiana. `L2` reducirá el tamaño de todos los pesos pero los hará 0. Si bien es menos eficiente en terminos de memoria, puede ser útil si queremos/necesitamos retener todos los parámetros.

$$
    \textit{TF-IDF}_{normalized} = \frac{tf \times idf}{\sqrt{(tf\times idf)^2}}
$$

Veamos como aplicarlo:

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(corpus)

In [12]:
vectors = vectors.todense()
vectors.shape

(4, 20)

> Notemos que cambiar la forma de vectorización en estos casos no cambia la longitud de nuestros vectores (que siempre está dada por la dimensionalidad del vocabulario, en este caso 20). Solo cambia los valores numericos que se asignan en los vectores.

In [14]:
vocab = vectorizer.get_feature_names_out()
pd.DataFrame(np.round(vectors, 2), columns=vocab)

,agua,común,congela,cuatro,de,del,el,en,es,estado,estados,grados,hielo,los,naturales,nombre,pura,se,sólido,uno
0,0.29,0.00,0.00,0.00,0.00,0.00,0.29,0.44,0.36,0.44,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.44,0.00
1,0.18,0.00,0.00,0.35,0.35,0.28,0.18,0.00,0.23,0.00,0.35,0.00,0.23,0.35,0.35,0.00,0.00,0.00,0.00,0.35
2,0.24,0.00,0.47,0.00,0.00,0.00,0.24,0.00,0.00,0.00,0.00,0.47,0.00,0.00,0.00,0.00,0.47,0.47,0.00,0.00
3,0.20,0.39,0.00,0.00,0.00,0.31,0.40,0.31,0.25,0.31,0.00,0.00,0.25,0.00,0.00,0.39,0.00,0.00,0.31,0.00


## Conclusión

En estos métodos, cada caractéritica o palabra que forma parte de nuestra entrada está representada en su propia dimensión, y como resultado, tenemos representaciones que dependen del tamaño del vocabulario. Estas representaciones tienen la característica de que la representación de cada palabra es independiente de las restantes. Esto quiere decir que el vector que corresponde a la palabra *perro* es tan distinto al vector que corresponde a *gato* como lo es al que corresponde con *heladera*.

En la siguiente sección veremos técnicas de vectorización de vectores de espacios continuos, donde estas características no se cumplen.